In [1]:
import pandas as pd
import tensorflow as tf

In [57]:
ratings = pd.read_csv('C://Projects//Movie Recommender System//Datasets//ratings_small.csv')

In [58]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [40]:
ratings.dropna(inplace=True)

In [59]:
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [60]:
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [61]:
from sklearn.model_selection import train_test_split

In [62]:
train_X,test_X = train_test_split(ratings, test_size=0.2)

In [63]:
train_X

,userId,movieId,rating,timestamp
53890,388,1293,4.5,1097626045
20715,141,648,3.0,979428119
37186,268,1206,3.0,1314894691
43370,311,153,3.0,898008136
30605,217,4306,5.0,1108160015
...,...,...,...,...
98078,655,32,4.0,1470074140
37222,268,1407,5.0,1314895088
95062,624,107945,2.0,1442162036
65769,468,597,2.5,1296195620


In [64]:
test_X

,userId,movieId,rating,timestamp
88458,587,3949,4.5,1131819438
73015,509,4066,3.0,1095897504
51027,378,58559,0.5,1443625495
45185,318,1,1.0,862902604
45360,321,1464,5.0,939399601
...,...,...,...,...
85546,574,56921,3.5,1232809970
87790,584,69122,4.5,1271763924
65062,463,2355,4.0,1049913431
16554,105,3584,3.5,1086094545


In [84]:
max_userid = ratings['userId'].drop_duplicates().max()
max_movieid = ratings['movieId'].drop_duplicates().max()

print(max_userid)
print(max_movieid)

671
163949


In [66]:
embedding_dimension = 32

In [86]:
#Movies Network
input_movies = tf.keras.layers.Input(shape=[1])
embed_movies = tf.keras.layers.Embedding(max_movieid + 1, embedding_dimension)(input_movies)
out_movies = tf.keras.layers.Flatten()(embed_movies)
#Users Network
input_users = tf.keras.layers.Input(shape=[1])
embed_users = tf.keras.layers.Embedding(max_userid + 1, embedding_dimension)(input_users)
out_users = tf.keras.layers.Flatten()(embed_users)

conc_layer = tf.keras.layers.Concatenate()([out_movies,out_users])
x = tf.keras.layers.Dense(128, activation='relu')(conc_layer)
x_out = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.Model([input_movies,input_users], x_out)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mean_squared_error')

In [87]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 1, 32)        5246400     input_19[0][0]                   
__________________________________________________________________________________________________
embedding_19 (Embedding)        (None, 1, 32)        21504       input_20[0][0]                   
____________________________________________________________________________________________

In [89]:
hist = model.fit([train_X.movieId,train_X.userId], train_X.rating,
                 batch_size=64,epochs=5,
                 verbose=1,
                 validation_data= ([test_X.movieId,test_X.userId], test_X.rating))

Epoch 1/5
1251/1251 [==============================] - 91s 73ms/step - loss: 0.7544 - val_loss: 0.8009
Epoch 2/5
1251/1251 [==============================] - 87s 70ms/step - loss: 0.7058 - val_loss: 0.7924
Epoch 3/5
1251/1251 [==============================] - 87s 69ms/step - loss: 0.6776 - val_loss: 0.7930
Epoch 4/5
1251/1251 [==============================] - 87s 69ms/step - loss: 0.6535 - val_loss: 0.8024
Epoch 5/5
1251/1251 [==============================] - 86s 69ms/step - loss: 0.6256 - val_loss: 0.8023
